In [ ]:
# KNN, 서포트백터머신, 렌덤포레스트, decision tree, GBM, logistic Regression

### KNN (분류)
- 새로 들어온 데이터 주변 k갸의 데이터의 class로 분류하는 기법 
- Euclidean 거리 계산

### suport  vector machin (분류)
- class간의 거리가 최대가 되도록 decision boundary를 만드는 기법

### decision tree  (분류)
- 독립변수의 조건에 따라 종속변수를 분리 
- 장점: 이해하기 쉬움 
- 단점: overfitting 
        
### random forest (분류)
- decison tree의 단점 overfiiting을 방지하기 위해 많은 수의 트리를 생성함으로써 overfitting이 예측에 큰 영향을 끼치지 못하도록 예방 (ensemble machine learning model)

### gradient boosting machine (분류/회귀)
- 부스팅 알고리즘. 순차적으로 학습, 예측하면서 잘못 예측한 데이터에 가중치를 부여하고 오류를 개선해 나가면서 학습하는 방식 
- 경사하강법을 이용하여 가중치 업데이트
- 예측성능 good, 수행시간 bad 

### logistic Regression (회귀)
- 독립변수와 종속변수가 선형적인 관계에 있다라는 가정하에 분석 
- 직선을 통해 종속변수를 예측하기때문에 독립변수의 중요도와 영향력을 파악하기 쉬움 


#### 앙상블 학습 --> 1. 배깅 2. 부스팅 
* 앙상블 학습 : 여러 개의 분류기를 생성하고, 그 예측을 결합함으로써 보다 정확한 예측을 도출
#### ----------------------------------------------------------------------------------------------

#### 배깅의 대표적인 모델 : 랜덤포레스트 / 부스팅의 대표적인 모델 : gradient
1. voting : 다른 ml 알고리즘이 같은 데이터 세트에 대해 학습하고 예측한 결과를 가지고 보팅을 통해 최종 예측 결과 선정
2. bagging : 단일 ml 알고리즘이 booststrapping 방식으로 샘플링된 데이터 세트에 대해서 학습을 통해 개별적인 예측을 수행한 결과를 보팅을 통해 최종 예측 결과 선정
3. bootstapping : 개별 분류기에게 데이터를 샘플링해서 추출하는 방식. 여러개의 데이터세트를 중첩되게 분리하는 것
* 교차검증이 데이터 세트간에 중첩 허용하지 않는 것과 다르게 bagging은 중첩 허용 
* 하이퍼 파라미터란, 일반적으로 머신러닝에서 어떠한 임의의 모델을 학습시킬때, 사람이 직접 튜닝 (설정) 해주어야하는 변수를 말한다.


In [1]:
import pandas as pd
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression # 로지스틱은 리니어 모델에서 파생 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

titanic_df= pd.read_csv('dataset/train.csv')
titanic_df.head(1)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NaN,S


In [2]:
# 일괄 전처리 사용자 함수(null 처리, 불필요 칼럼 삭제, 레이블 인코딩) 
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Null 처리 함수
# Age(평균), Cabin('N'), Embarked('N'), Fare(0)
def fillna(df):
    df['Age'].fillna(df['Age'].mean(),inplace=True)
    df['Cabin'].fillna('N',inplace=True)
    df['Embarked'].fillna('N',inplace=True)
    df['Fare'].fillna(0,inplace=True)
    return df

# 머신러닝 알고리즘에 불필요한 속성 제거
# PassengerId, Name, Ticket(티켓번호)
def drop_features(df):
    df.drop(['PassengerId','Name','Ticket'],axis=1,inplace=True)
    return df

# 레이블 인코딩 수행.
# Cabin(선실번호 첫문자만 추출 후 인코딩), Sex(성별), Embarked(중간 정착 항구)
def format_features(df):
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Cabin','Sex','Embarked']
    for feature in features:
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

# 앞에서 설정한 Data Preprocessing 함수 호출
def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

In [3]:
y_titanic_df = titanic_df['Survived']
X_titanic_df = titanic_df.drop('Survived', axis=1)
X_titanic_df = transform_features(X_titanic_df)

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic_df,
                                                  test_size=0.2,
                                                  random_state=11)


In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

dt_clf = DecisionTreeClassifier(random_state=5)
rf_clf = RandomForestClassifier(random_state=5)
lr_clf = LogisticRegression()
nc_clf = KNeighborsClassifier(n_neighbors = 5)
svm_clf = SVC(random_state=5)
gbm_clf = GradientBoostingClassifier(random_state=5)

In [10]:
# DecisionTreeClassifier
dt_clf.fit(X_train, y_train)
dt_pred = dt_clf.predict(X_test)
class_name= dt_clf.__class__.__name__
print('{0} 정확도:{1:.4f}'.format(class_name,accuracy_score(y_test,dt_pred)))


DecisionTreeClassifier 정확도:0.7989


In [11]:
# RandomForeat
rf_clf.fit(X_train, y_train)
rf_pred = rf_clf.predict(X_test)
class_name= rf_clf.__class__.__name__
print('{0} 정확도:{1:.4f}'.format(class_name, accuracy_score(y_test, rf_pred)))



RandomForestClassifier 정확도:0.8659


In [12]:
# LogisiticRegression
lr_clf.fit(X_train, y_train)
lr_pred = lr_clf.predict(X_test)
class_name= lr_clf.__class__.__name__
print('{0} 정확도:{1:.4f}'.format(class_name, accuracy_score(y_test,lr_pred)))


LogisticRegression 정확도:0.8492


In [13]:
# KNeighborsClssifier 
nc_clf.fit(X_train, y_train)
nc_pred = nc_clf.predict(X_test)
class_name= nc_clf.__class__.__name__
print('{0} 정확도:{1:.4f}'.format(class_name, accuracy_score(y_test,nc_pred)))



KNeighborsClassifier 정확도:0.7430


In [14]:
# Suport Vector Machine
svm_clf.fit(X_train, y_train)
svm_pred = svm_clf.predict(X_test)
class_name= svm_clf.__class__.__name__
print('{0} 정확도:{1:.4f}'.format(class_name, accuracy_score(y_test,svm_pred)))

SVC 정확도:0.7207


In [15]:
# gradient Boosting Machine 
gbm_clf.fit(X_train, y_train)
gbm_pred = gbm_clf.predict(X_test)
class_name= gbm_clf.__class__.__name__
print('{0} 정확도:{1:.4f}'.format(class_name, accuracy_score(y_test, gbm_pred)))

GradientBoostingClassifier 정확도:0.8715
